<a href="https://colab.research.google.com/github/Dhanush-sai-reddy/langchain/blob/main/1semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
pip install langchain-community pypdf

In [74]:
from langchain_core.documents import Document
#sample from docs
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [75]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "FLAP-SAM.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

15


In [76]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

A Federated Learning-Friendly Approach for
Parameter-Efficient Fine-Tuning of SAM in
3D Segmentation
Mothilal Asokan , Joseph Geo Benjamin (/enve♀e), Mohammad Yaqub , and
Karthik Nandakumar
Mohamed bi

{'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-08-01T00:50:30+00:00', 'author': '', 'keywords': '', 'moddate': '2024-08-01T00:50:30+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'FLAP-SAM.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}


In [77]:
#splittin
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=880,
    chunk_overlap=140,##600
    add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print(len(all_splits))

55


In [78]:
#embeddings

In [79]:
pip install -U langchain-google-genai google-generativeai


In [80]:
pip install -U "langchain-openai"

In [62]:
pip install -U langchain-huggingface sentence-transformers


In [81]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"

)

vector = embeddings.embed_query(
    "Federated learning with SAM for 3D segmentation"
)

print(len(vector))


384


In [82]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 384

[0.0038401151541620493, -0.07868853956460953, -0.007508019916713238, -0.03382249176502228, 0.045316170901060104, -0.06627873331308365, 0.007356824818998575, 0.005070831160992384, 0.005041068885475397, -0.030346399173140526]


In [83]:
from langchain_core.vectorstores import InMemoryVectorStore

In [84]:
vector_store = InMemoryVectorStore(embeddings)#we specify to use this embeddings model to convert texts

In [85]:
ids = vector_store.add_documents(documents=all_splits)

In [86]:
ids[0]#identifier

'1f9d705f-a7e1-49be-955d-c0dc97e8d11d'

In [87]:
results = vector_store.similarity_search(
    "segmentation"
)

print(results[3])

page_content='are relevant to the content of this article.
References
1. Bloch, N., Madabhushi, A., Huisman, H., Freymann, J., Kirby, J., Grauer, M.,
Enquobahrie, A., Jaffe, C., Clarke, L., Farahani, K.: Nci-isbi 2013 challenge: auto-
mated segmentation of prostate structures. The Cancer Imaging Archive 370(6),
5 (2015)
2. Chen, C., Miao, J., Wu, D., Yan, Z., Kim, S., Hu, J., Zhong, A., Liu, Z., Sun,
L., Li, X., et al.: Ma-sam: Modality-agnostic sam adaptation for 3d medical image
segmentation. arXiv preprint arXiv:2309.08842 (2023)
3. Chen, G., Liu, F., Meng, Z., Liang, S.: Revisiting parameter-efficient tuning: Are
we really there yet? In: Proceedings of the 2022 Conference on Empirical Methods
in Natural Language Processing. pp. 2612–2626 (2022)
4. Cheng, D., Qin, Z., Jiang, Z., Zhang, S., Lao, Q., Li, K.: Sam on medical images:' metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-08-01T00:50:30+00:00', 'author': '', 'keywords': '', 'modda

In [88]:

results = vector_store.similarity_search_with_score("What is LoRA")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.420645681189624

page_content='partial decoder fine tuning (PDecFT). Since only a small fraction of parameters,
namely, θMD-UP, and θMD-HYP are updated, PDecFT has high parameter efficiency
but usually provides only sub-optimal performance.
LoRA adapters: Low-rank adaptation [9] is a promising PEFT technique
widely used for adapting foundation models to downstream tasks. Each atten-
tion layer within a transformer block has four weight (projection) matrices Wq
(query), Wk (key), Wv (value), and Wo (output), where each W ∈ Rd×d′
. The
core idea of LoRA is to constrain the modifications to a pre-trained weight ma-
trix W to a linear update matrix ∆W, which can be further constrained using
a low-rank decomposition, i.e., ∆W = BA, where B ∈ Rd×r, A∈ Rr×d′
and
the rank r ≪ min{d, d′}. This approach effectively reduces the parameter space' metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-08-01T00:50:30+00:00', 'author': '', 'keywords': 

In [89]:
embedding = embeddings.embed_query("dataset used")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='A Federated Learning-Friendly Approach for
Parameter-Efficient Fine-Tuning of SAM in
3D Segmentation
Mothilal Asokan , Joseph Geo Benjamin (/enve♀e), Mohammad Yaqub , and
Karthik Nandakumar
Mohamed bin Zayed University of Artificial Intelligence (MBZUAI),
Abu Dhabi, United Arab Emirates
{mothilal.asokan, joseph.benjamin, mohammad.yaqub,
karthik.nandakumar}@mbzuai.ac.ae
Abstract. Adapting foundation models for medical image analysis re-
quires finetuning them on a considerable amount of data because of ex-
treme distribution shifts between natural (source) data used for pretrain-
ing and medical (target) data. However, collecting task-specific medical
data for such finetuning at a central location raises many privacy con-
cerns. Although Federated learning (FL) provides an effective means for
training on private decentralized data, communication costs in federat-' metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-08-01T00:50:30

####well here we move to retrivers part

In [72]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "Cost reduction by FLAP-SAM",
        "what  is Lora?",
    ],
)[0]
#since we used a hugging face embeddings model of 384 d,it may not work well as /for a semantic search

[Document(id='34fea752-b9c4-46ae-af46-d8959bb5f369', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-08-01T00:50:30+00:00', 'author': '', 'keywords': '', 'moddate': '2024-08-01T00:50:30+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'FLAP-SAM.pdf', 'total_pages': 15, 'page': 13, 'page_label': '14', 'start_index': 1481}, page_content='FLAP-SAM (Ours)\n{θLoRA , θUP , θHYP }\n(1.712M /91.767M) 1.9% Centralized 0.9854 0.9864 0.9835 0.9851\nMA-SAM ‡\n(28.667M /115.297M) 25%\nCentralized 0.972 0.972 0.968 0.971')]